## <B>Normalize Gaze Coordinate Data</B>

In [ ]:
# 필요한 라이브러리 불러오기
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

name = "37"

movies = ["기생충1", "기생충2", "낭닥1", "낭닥2", "코1", "코2",  "마녀1", "마녀2", "레버1", "레버2"]

for movie in movies:
    # 데이터 파일 불러오기
    # 파일명: 영상_좌표_보정.csv
    data = pd.read_csv(f'{name} {movie}.csv')
    
    # 열 이름 확인 및 공백 제거
    data.columns = data.columns.str.strip()  # 열 이름의 불필요한 공백 제거
    
    # X와 Y 컬럼 확인
    print("원본 데이터:")
    print(data.head())
    
    # Min-Max Scaler를 사용하여 정규화 (범위: -1에서 1)
    scaler = MinMaxScaler(feature_range=(-1, 1))
    
    # X와 Y를 정규화
    data[['X', 'Y']] = scaler.fit_transform(data[['X', 'Y']])
    
    # 결과 확인
    print("\n정규화된 데이터:")
    print(data.head())
    
    # 정규화된 데이터를 새로운 CSV 파일로 저장
    data.to_csv(f'{name} {movie}(정규화).csv', index=False)

## <B>Presence of eyes in blurred areas of experimental footage O/X</B>

In [ ]:
import pandas as pd
import warnings

# 경고를 무시하도록 설정
warnings.filterwarnings("ignore")

# OX 분석 및 통합 처리 함수 정의
def process_and_analyze(name, movie, box_file):
    try:
        # 데이터 읽기
        points_df = pd.read_csv(f'{name} {movie}(정규화).csv')  # 정규화된 점 데이터
        boxes_df = pd.read_csv(box_file)  # 박스 데이터

        # Timestamp 변환 및 초기화
        points_df['Timestamp (Current Time)'] = pd.to_datetime(points_df['Timestamp (Current Time)'], errors='coerce')
        points_df.dropna(subset=['Timestamp (Current Time)'], inplace=True)  # NaT 삭제

        # 기준 시간 설정 및 초 단위 변환
        base_time = points_df['Timestamp (Current Time)'].iloc[0]
        points_df['Time (seconds)'] = (points_df['Timestamp (Current Time)'] - base_time).dt.total_seconds()

        # 시간 값 반올림 및 중복 제거
        points_df['Time (seconds)'] = points_df['Time (seconds)'].round(1)
        boxes_df['Time (seconds)'] = boxes_df['Time (seconds)'].round(1)
        points_df = points_df.drop_duplicates(subset='Time (seconds)', keep='first')
        boxes_df = boxes_df.drop_duplicates(subset='Time (seconds)', keep='first')

        # 시간 기준 병합
        merged_df = pd.merge(boxes_df, points_df, how='left', on='Time (seconds)')
        merged_df.to_csv(f'{name}_{movie}_merged.csv', index=False)  # 병합 결과 저장

        # OX 결과 계산
        def point_box_with_margin(row):
            expanded_x1 = row['x1'] - 0.25
            expanded_y1 = row['y1'] - 0.25
            expanded_x2 = row['x2'] + 0.25
            expanded_y2 = row['y2'] + 0.25
            return "O" if expanded_x1 <= row['X'] <= expanded_x2 and expanded_y1 <= row['Y'] <= expanded_y2 else "X"

        merged_df['Result'] = merged_df.apply(point_box_with_margin, axis=1)

        # O 비율 계산 및 출력
        o_count = merged_df['Result'].value_counts().get('O', 0)
        total_count = len(merged_df)
        o_ratio = o_count / total_count if total_count > 0 else 0

        print(f'"{movie}" 분석 결과:')
        print(f'  "O"의 개수: {o_count}')
        print(f'  "O"의 비율: {o_ratio:.2%}')

        # 최종 결과 저장
        merged_df.to_csv(f'{name} {movie}_result.csv', index=False)

    except FileNotFoundError as e:
        print(f"파일을 찾을 수 없습니다: {e}")
    except Exception as e:
        print(f"오류 발생: {e}")

# 실행 예시
name = "37"
movie_box_map = {
    "기생충1": "영화1 보정.csv",
    "기생충2": "영화1 보정.csv",
    "낭닥1": "영화2 보정.csv",
    "낭닥2": "영화2 보정.csv",
    "코1": "영화3 보정.csv",
    "코2": "영화3 보정.csv",
    "마녀1": "영화4 보정.csv",
    "마녀2": "영화4 보정.csv",
    "레버1": "영화5 보정.csv",
    "레버2": "영화5 보정.csv",
}

for movie, box_file in movie_box_map.items():
    process_and_analyze(name, movie, box_file)